In [ ]:
import pandas as pd
import datetime as dt

In [ ]:
def player_snap_clean(Start_Year,End_Year):
    dfname = []
    try:
        for num in range(Start_Year,End_Year+1):
            year = Start_Year
            dfname.append(f'{num}_Offense')
            dfname.append(f'{num}_Defense')
            data = {key: pd.read_csv(f'FantasyPros_Fantasy_Football_{key}_Snap_Counts.csv') for key in dfname}

        while year < End_Year+1:
            for key in data:
                data[key].insert(loc=3,column='Year',value=year,allow_duplicates=True)
                data[key]['Year'] = data[key]['Year'].astype(int)
                year += 0.5

        for key in data:
            data[key].drop(['TTL','AVG'],axis=1,inplace=True)
            data[key] = pd.melt(data[key],id_vars=['Player','Pos','Team','Year'],\
                      value_vars=['1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17'])
            data[key].rename(columns={'variable':'Week','value':'Snaps'},inplace=True)
            data[key].replace({'Team':\
                      {'GB':'GNB','JAC':'JAX','KC':'KAN','NE':'NWE','NO':'NOR','SF':\
                       'SFO','TB':'TAM','Multi':None,'LV':'OAK',},'Snaps':{'bye':None}},inplace=True)
            data[key].dropna(thresh=3,inplace=True)

        player = pd.concat(data.values(), ignore_index=True)

        team = pd.read_csv('nflteamsnaps.csv')
        team.drop(['LTime','Y/P'],axis=1,inplace=True)
        team.rename(columns={'Unnamed: 6':'Away_Home','Time.1':'ToG'},inplace=True)
        team['Date'] = pd.to_datetime(team['Date'])
        team['ToG'] = pd.to_datetime(team['ToG'])
        team['Away_Home'].fillna(value='Home',inplace=True)
        team['Away_Home'].replace('@','Away',inplace=True)
        team['Away_Home'].replace('nan','Home',inplace=True)
        team['TO'].fillna(value=0,inplace=True)
        team.insert(loc=2,column='Month',value=team['Date'].dt.month)
        df_bridge = team['ToP'].str.split(":",expand=True)
        team['ToP'] = (df_bridge[0].astype(int)*60)+df_bridge[1].astype(int)
        team['Week'] = team['Week'].astype(str)

        player_snaps = pd.merge(left=player,right=team,how='outer',left_on=['Team','Year','Week'],right_on=['Tm','Year','Week'])

        print(player)
        print(team)
        print(player_snaps)
        player_snaps.to_csv(f'player_snaps_{Start_Year}_{End_Year}.csv',index=False)
        
        
    
    except:
        print('Please make sure the labels are "Starting Year" and "Ending Year" in that order and they are not \nout of range of what is in the data. If you are still receiving and error, please consult the code or \nmake sure the columns are identical. All data must be pulled from FantasyPros.')

In [ ]:
player_snap_clean(2017,2019)